In [26]:
# https://wikidocs.net/233348

In [27]:
# pip install langchain

In [28]:
# pip install -U langchain-ollama

In [29]:
# pip install langchain_teddynote
#  ollama create llama-3.1-70b-instruct-lorablated.Q4_K_M:latest -f Modelfile

In [30]:

import warnings
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [31]:
df = pd.read_csv('../../data/q_output.csv')

In [32]:
llm = OllamaLLM(model="llama-3.1-70b-instruct-lorablated.Q4_K_M:latest")
# llm = OllamaLLM(model="llama3.1:70b")


In [33]:
df.shape[0]

44

In [34]:
# df=df[['question', 'answer']]

In [35]:
population = np.arange(44)
np.random.seed(1111)
fewshot_q_id = np.random.choice(population, size=3, replace=False)


In [36]:
eval_q_id =np.setdiff1d(population, fewshot_q_id)

In [37]:
examples = []
for idx in fewshot_q_id:
# for idx in [19]:
    temp_dict = {"question" : str(df.loc[idx, 'question']),
                 "answer"   : str(df.loc[idx, 'answer'])}
    examples.append(temp_dict)

In [38]:
example_prompt = PromptTemplate.from_template(
    "Question: {question} *||* Answer:{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Here is the question about software development. The title of the question is "Strange error: TypeError: forward() takes 2 positional arguments but 3 were given". and body of the question is "I am trying to train a VAE model. However I keep getting the following error:




I tried many combinations of arguments but nothing seems to work. Can you spot the error?
As you can see in the following model, the forward function was called in the train() function with only two arguments and not 3. Also the error hints at the line self.encoder(x, edge_index). But when I try to remove one of these arguments, I get an error saying not enough arguments.
Here is my model:




and here is my train function:




Here is the code I am using to train the model:




". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question *||* Answer:Difficulty class : Intermediate Level  | General rule set : I1-Advanced Features | Granular breakdown 

In [39]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question} *||* Answer:",
    input_variables=["question"],
)

In [40]:
eval_result = pd.DataFrame(columns = ['id', 'result'])
for idx in tqdm(eval_q_id):
    question = df.loc[idx, 'question']
    final_prompt = prompt.format(question=question)
    chain = prompt | llm

    # chain 호출
    response = chain.invoke({"question": question})
    tmp_dict = {'id' : df.loc[idx, 'id']
                ,'result' : response}
    eval_result = pd.concat([eval_result, pd.DataFrame([tmp_dict])], ignore_index=True)

100%|██████████| 41/41 [02:20<00:00,  3.43s/it]


In [41]:
result_df = pd.merge(df,eval_result, on = 'id' )

In [42]:
result_df.head()

,Unnamed: 0,id,question,answer,result
0,0,74627211,Here is the question about software developmen...,Difficulty class : Intermediate Level | Genera...,Difficulty class : Intermediate Level | Gener...
1,1,74700764,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Basic Level Knowledge | Gen...
2,2,74727945,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Intermediate Level | Gener...
3,3,74793352,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Basic Level Knowledge | Gen...
4,4,74936129,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Intermediate Level | Gener...


In [43]:
result_df['answer_diff_class'] = [x[1] for x in result_df['answer'].str.split(r"[|:]+")]
result_df['answer_general_rs'] = [x[3] for x in result_df['answer'].str.split(r"[|:]+")]
result_df['answer_grenular_bd'] = [x[5] for x in result_df['answer'].str.split(r"[|:]+")]


In [44]:
result_df['result_diff_class']  = [x[1] for x in result_df['result'].str.split(r"[|:]+")]
result_df['result_general_rs']  = [x[3] for x in result_df['result'].str.split(r"[|:]+")]
result_df['result_grenular_bd'] = [x[5] for x in result_df['result'].str.split(r"[|:]+")]


In [45]:
result_df.head()

,Unnamed: 0,id,question,answer,result,answer_diff_class,answer_general_rs,answer_grenular_bd,result_diff_class,result_general_rs,result_grenular_bd
0,0,74627211,Here is the question about software developmen...,Difficulty class : Intermediate Level | Genera...,Difficulty class : Intermediate Level | Gener...,Intermediate Level,I3-Frameworks,웹 프레임워크/ORM,Intermediate Level,I3-Frameworks,웹프레임워크/ORM
1,1,74700764,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Basic Level Knowledge | Gen...,Basic Level Knowledge,B1-Syntax,기본 문법/타입 변환,Basic Level Knowledge,B1-Syntax,기본 문법/타입 변환
2,2,74727945,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Intermediate Level | Gener...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Intermediate Level,I2-Performance,성능 최적화/데이터 구조
3,3,74793352,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Basic Level Knowledge | Gen...,Basic Level Knowledge,B3-Common Problems,에러 처리/파일 처리,Basic Level Knowledge,B1-Syntax,기본 문법/타입 변환
4,4,74936129,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Intermediate Level | Gener...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Intermediate Level,I3-Frameworks,웹 프레임워크/ORM


In [46]:
result_df['answer_diff_class'] = result_df['answer_diff_class'].str.strip()
result_df['result_diff_class'] = result_df['result_diff_class'].str.strip()

In [47]:
result_df['answer_diff_class_0'] = [x[0] for x in result_df['answer_diff_class'].str.split(' ')]
result_df['result_diff_class_0'] = [x[0] for x in result_df['result_diff_class'].str.split(' ')]

In [48]:
result_df['equal_yn'] = np.where(result_df['answer_diff_class_0']==result_df['result_diff_class_0'], 1, 0)

In [49]:
acc = (result_df['equal_yn'].sum()/result_df.shape[0])*100
print(acc)

68.29268292682927


In [50]:
result_df[result_df['equal_yn'] ==0]

,Unnamed: 0,id,question,answer,result,answer_diff_class,answer_general_rs,answer_grenular_bd,result_diff_class,result_general_rs,result_grenular_bd,answer_diff_class_0,result_diff_class_0,equal_yn
5,5,75022065,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Intermediate Level | Gener...,Basic Level Knowledge,B1-Syntax,기본 문법/타입 변환,Intermediate Level,I2-Performance,성능 최적화/데이터 구조,Basic,Intermediate,0
7,8,75259252,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Basic Level Knowledge | Gen...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Basic Level Knowledge,B3-Common Problems,에러 핸들링/파일 연산,Intermediate,Basic,0
10,11,75333570,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Basic Level Knowledge | Gen...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Basic Level Knowledge,B3-Common Problems,에러 핸들링/파일 연산,Intermediate,Basic,0
12,13,75492157,Here is the question about software developmen...,Difficulty class : Advanced Level Knowledge | ...,Difficulty class : Intermediate Level | Gener...,Advanced Level Knowledge,A3-Data & AI,ML/빅데이터/스트림처리,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Advanced,Intermediate,0
13,14,75524574,Here is the question about software developmen...,Difficulty class : Advanced Level Knowledge | ...,Difficulty class : Intermediate Level | Gener...,Advanced Level Knowledge,A2-System Architecture,분산시스템/확장성/보안,Intermediate Level,I3-Frameworks,웹 프레임워크/ORM,Advanced,Intermediate,0
14,15,75711831,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Basic Level Knowledge | Gen...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Basic Level Knowledge,B3-Common Problems,에러 핸들링/파일 연산,Intermediate,Basic,0
20,21,75965577,Here is the question about software developmen...,Difficulty class : Advanced Level Knowledge | ...,Difficulty class : Intermediate Level | Gener...,Advanced Level Knowledge,A3-Data & AI,ML/빅데이터/스트림처리,Intermediate Level,I2-Performance,성능 최적화/데이터 구조,Advanced,Intermediate,0
22,24,76059210,Here is the question about software developmen...,Difficulty class : Basic Level Knowledge | Gen...,Difficulty class : Intermediate Level | Gener...,Basic Level Knowledge,B3-Common Problems,에러 처리/파일 처리,Intermediate Level,I2-Performance,성능 최적화/데이터 구조,Basic,Intermediate,0
23,25,76115814,Here is the question about software developmen...,Difficulty class : Intermediate Level | Gener...,Difficulty class : Basic Level Knowledge | Gen...,Intermediate Level,I1-Advanced Features,데코레이터/제너레이터/OOP,Basic Level Knowledge,B3-Common Problems,에러 핸들링/파일 연산,Intermediate,Basic,0
26,28,76427264,Here is the question about software developmen...,Difficulty class : Advanced Level Knowledge | ...,Difficulty class : Basic Level Knowledge | Gen...,Advanced Level Knowledge,A2-System Architecture,분산시스템/확장성/보안,Basic Level Knowledge,B2-Environment,패키지 관리/가상 환경,Advanced,Basic,0


df